In [1]:
import numpy as np
from copy import deepcopy
import random
from typing import Any, Tuple, Optional, Sequence

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred.evaluate import Evaluator

In [5]:
import sys
sys.path.append("..")
from dataset import temp_partition_arxiv

In [6]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')

In [7]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [8]:
class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x
    

class ThreeLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, edge_index)
        return x
    

    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x1 = self.linear1(x)
        x1 = F.elu(x1)
        x1 = F.dropout(x1, p=self.dropout)
        output = self.linear2(x1)
        # x = F.softmax(x, dim=1)
        features = x1
        return output, features

In [9]:
def train(encoder, mlp, optimizer, data):
    encoder.train()
    mlp.train()
    
    out, _ = mlp(encoder(data.x, data.edge_index))
    out = F.log_softmax(out, dim=1)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()
    
@torch.no_grad()
def test(encoder, mlp, data, evaluator):
    encoder.eval()
    mlp.eval()
    
    out, _ = mlp(encoder(data.x, data.edge_index))
    out = F.log_softmax(out, dim=1)
    val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask]).item()
    y_pred = out.argmax(dim=-1, keepdim=True)
    val_acc = evaluator.eval({
        'y_true': data.y[data.val_mask].unsqueeze(1),
        'y_pred': y_pred[data.val_mask],
    })['acc']
    
    return val_loss, val_acc

In [10]:
def pseudo_label(encoder, classifier, tgt_data):
    pseudo_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
    pseudo_y = F.log_softmax(pseudo_y, dim=1).argmax(dim=-1, keepdim=False)
    return pseudo_y

def adapt_train(encoder, classifier, src_data, tgt_data, optimizer):
    encoder.train()
    classifier.train()
    pseudo_tgt_label = pseudo_label(encoder, classifier, tgt_data)
    src_y, _ = classifier(encoder(src_data.x, src_data.edge_index))
    tgt_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
    src_loss = F.nll_loss(F.log_softmax(src_y[src_data.train_mask], dim=1), src_data.y[src_data.train_mask])
    tgt_loss = F.nll_loss(F.log_softmax(tgt_y[tgt_data.train_mask], dim=1), pseudo_tgt_label[tgt_data.train_mask])
    loss = src_loss + tgt_loss 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), src_loss.item(), tgt_loss.item()
    
@torch.no_grad()
def adapt_val(encoder, classifier, src_data, tgt_data):
    encoder.eval()
    classifier.eval()
    pseudo_tgt_label = pseudo_label(encoder, classifier, tgt_data)
    src_y, _ = classifier(encoder(src_data.x, src_data.edge_index))
    tgt_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
    src_loss = F.nll_loss(F.log_softmax(src_y[src_data.val_mask], dim=1), src_data.y[src_data.val_mask])
    tgt_loss = F.nll_loss(F.log_softmax(tgt_y[tgt_data.val_mask], dim=1), pseudo_tgt_label[tgt_data.val_mask])
    loss = src_loss + tgt_loss
    return loss.item(), src_loss.item(), tgt_loss.item()

## Load Data ##

In [11]:
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index, data.num_nodes) # mimicking barlow twins repo

## Data Partition ##

* Train: 0-2011
* Val: 2012
* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)
** 2019-2020

## Source Training Stage ##

* Train: 0-2011
* Val: 2012

In [12]:
feat_dim = data.x.shape[1]
class_dim = data.y
hidden_dim = 128
emb_dim = 256
encoder = TwoLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 40)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 500

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

In [13]:
data_2012_2013 = temp_partition_arxiv(data, year_bound=[-1,2012,2013], proportion=1.0)
data_2012_2013 = data_2012_2013.to(device)

In [14]:
best_acc = 0
best_encoder = None
best_mlp = None
evaluator = Evaluator(name='ogbn-arxiv')
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, data_2012_2013)
    val_loss, val_acc = test(encoder, mlp, data_2012_2013, evaluator)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val Acc:{round(val_acc, 4)}")
    if val_acc > best_acc:
        best_acc = val_acc
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

encoder = deepcopy(best_encoder)
mlp = deepcopy(best_mlp)

Epoch:1/500 Train Loss:3.7146 Val Loss:3.6349 Val Acc:0.1565
Epoch:2/500 Train Loss:3.6308 Val Loss:3.5643 Val Acc:0.1977
Epoch:3/500 Train Loss:3.5491 Val Loss:3.4867 Val Acc:0.2036
Epoch:4/500 Train Loss:3.4588 Val Loss:3.4016 Val Acc:0.2045
Epoch:5/500 Train Loss:3.3554 Val Loss:3.3091 Val Acc:0.2045
Epoch:6/500 Train Loss:3.2474 Val Loss:3.2373 Val Acc:0.2045
Epoch:7/500 Train Loss:3.159 Val Loss:3.2137 Val Acc:0.2045
Epoch:8/500 Train Loss:3.1197 Val Loss:3.2375 Val Acc:0.2045
Epoch:9/500 Train Loss:3.1278 Val Loss:3.2435 Val Acc:0.2045
Epoch:10/500 Train Loss:3.1227 Val Loss:3.1977 Val Acc:0.2045
Epoch:11/500 Train Loss:3.0835 Val Loss:3.1572 Val Acc:0.2034
Epoch:12/500 Train Loss:3.0444 Val Loss:3.1318 Val Acc:0.2026
Epoch:13/500 Train Loss:3.014 Val Loss:3.1054 Val Acc:0.1984
Epoch:14/500 Train Loss:3.0002 Val Loss:3.1003 Val Acc:0.1997
Epoch:15/500 Train Loss:2.985 Val Loss:3.0831 Val Acc:0.2009
Epoch:16/500 Train Loss:2.9754 Val Loss:3.0729 Val Acc:0.2068
Epoch:17/500 Train L

Epoch:143/500 Train Loss:1.5089 Val Loss:1.7335 Val Acc:0.511
Epoch:144/500 Train Loss:1.5028 Val Loss:1.7282 Val Acc:0.5117
Epoch:145/500 Train Loss:1.5014 Val Loss:1.7215 Val Acc:0.5094
Epoch:146/500 Train Loss:1.4982 Val Loss:1.7283 Val Acc:0.5172
Epoch:147/500 Train Loss:1.5045 Val Loss:1.7207 Val Acc:0.5159
Epoch:148/500 Train Loss:1.4934 Val Loss:1.7122 Val Acc:0.5207
Epoch:149/500 Train Loss:1.4897 Val Loss:1.7171 Val Acc:0.5167
Epoch:150/500 Train Loss:1.4878 Val Loss:1.7082 Val Acc:0.5201
Epoch:151/500 Train Loss:1.4852 Val Loss:1.7078 Val Acc:0.5161
Epoch:152/500 Train Loss:1.4839 Val Loss:1.7085 Val Acc:0.5173
Epoch:153/500 Train Loss:1.4802 Val Loss:1.7029 Val Acc:0.5253
Epoch:154/500 Train Loss:1.4737 Val Loss:1.705 Val Acc:0.5186
Epoch:155/500 Train Loss:1.474 Val Loss:1.7022 Val Acc:0.5197
Epoch:156/500 Train Loss:1.4681 Val Loss:1.6938 Val Acc:0.5277
Epoch:157/500 Train Loss:1.4676 Val Loss:1.6962 Val Acc:0.5221
Epoch:158/500 Train Loss:1.4619 Val Loss:1.6918 Val Acc:0.

Epoch:277/500 Train Loss:1.2404 Val Loss:1.5355 Val Acc:0.5667
Epoch:278/500 Train Loss:1.2371 Val Loss:1.5374 Val Acc:0.5652
Epoch:279/500 Train Loss:1.2351 Val Loss:1.5383 Val Acc:0.5649
Epoch:280/500 Train Loss:1.236 Val Loss:1.5319 Val Acc:0.5652
Epoch:281/500 Train Loss:1.2323 Val Loss:1.5374 Val Acc:0.5649
Epoch:282/500 Train Loss:1.2312 Val Loss:1.5346 Val Acc:0.5672
Epoch:283/500 Train Loss:1.2324 Val Loss:1.5378 Val Acc:0.5697
Epoch:284/500 Train Loss:1.2283 Val Loss:1.5345 Val Acc:0.5644
Epoch:285/500 Train Loss:1.2241 Val Loss:1.5422 Val Acc:0.5692
Epoch:286/500 Train Loss:1.2288 Val Loss:1.5324 Val Acc:0.5719
Epoch:287/500 Train Loss:1.2237 Val Loss:1.5325 Val Acc:0.5716
Epoch:288/500 Train Loss:1.2263 Val Loss:1.5329 Val Acc:0.5709
Epoch:289/500 Train Loss:1.2195 Val Loss:1.537 Val Acc:0.572
Epoch:290/500 Train Loss:1.2249 Val Loss:1.5207 Val Acc:0.574
Epoch:291/500 Train Loss:1.2193 Val Loss:1.5284 Val Acc:0.5717
Epoch:292/500 Train Loss:1.219 Val Loss:1.547 Val Acc:0.569

Epoch:424/500 Train Loss:1.0951 Val Loss:1.4919 Val Acc:0.5851
Epoch:425/500 Train Loss:1.0918 Val Loss:1.5123 Val Acc:0.5754
Epoch:426/500 Train Loss:1.098 Val Loss:1.4917 Val Acc:0.584
Epoch:427/500 Train Loss:1.0969 Val Loss:1.495 Val Acc:0.5824
Epoch:428/500 Train Loss:1.0977 Val Loss:1.5085 Val Acc:0.582
Epoch:429/500 Train Loss:1.0968 Val Loss:1.5057 Val Acc:0.581
Epoch:430/500 Train Loss:1.0992 Val Loss:1.5102 Val Acc:0.5823
Epoch:431/500 Train Loss:1.0925 Val Loss:1.5002 Val Acc:0.5848
Epoch:432/500 Train Loss:1.0905 Val Loss:1.5101 Val Acc:0.5765
Epoch:433/500 Train Loss:1.0896 Val Loss:1.5148 Val Acc:0.5773
Epoch:434/500 Train Loss:1.0907 Val Loss:1.5079 Val Acc:0.5812
Epoch:435/500 Train Loss:1.0899 Val Loss:1.504 Val Acc:0.5812
Epoch:436/500 Train Loss:1.0882 Val Loss:1.5054 Val Acc:0.5837
Epoch:437/500 Train Loss:1.0912 Val Loss:1.5079 Val Acc:0.581
Epoch:438/500 Train Loss:1.0877 Val Loss:1.519 Val Acc:0.5737
Epoch:439/500 Train Loss:1.0879 Val Loss:1.5084 Val Acc:0.5843


In [15]:
best_acc

0.5866355866355867

# Prequential Evaluation at Subsequent Time Steps #

In [16]:
def continual_adapt(src_data, tgt_split, encoder, mlp, device, lr=1e-3):
    print("Start partitioning data...")
    tgt_data = temp_partition_arxiv(data, year_bound=tgt_split, proportion=1.0)
    tgt_data.to(device)
    print("Finish partitioning data...")
    tgt_encoder, tgt_mlp = deepcopy(encoder), deepcopy(mlp)
    tgt_optimizer = torch.optim.Adam(list(tgt_encoder.parameters()) + list(tgt_mlp.parameters()), lr=lr)
    
    epochs = 500
    best_val_loss = np.inf
    best_tgt_encoder, best_tgt_mlp = None, None
    patience = 10
    staleness = 0

    for e in range(1, epochs + 1):
        train_loss, train_src_loss, train_tgt_loss = adapt_train(tgt_encoder, tgt_mlp, src_data, tgt_data, tgt_optimizer)
        val_loss, val_src_loss, val_tgt_loss = adapt_val(tgt_encoder, tgt_mlp, src_data, tgt_data)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_tgt_encoder = deepcopy(tgt_encoder)
            best_tgt_mlp = deepcopy(tgt_mlp)
            staleness = 0
        else:
            staleness += 1
        print(f'Epoch {e}/{epochs} Train Loss: {round(train_loss,5)} Train Src Loss: {round(train_src_loss,5)} Train Tgt Loss: {round(train_tgt_loss,5)} \n \
                Val Loss: {round(val_loss,5)} Val Src Loss: {round(val_src_loss,5)} Val Tgt Loss: {round(val_tgt_loss,5)}')
        if staleness > patience:
            break

    tgt_encoder = deepcopy(best_tgt_encoder)
    tgt_mlp = deepcopy(best_tgt_mlp)
    
    return tgt_encoder, tgt_mlp, best_val_loss

In [17]:
lr = 1e-3
test_acc_list = []

## 2013-2014 ##

* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)

In [18]:
data_2013_2015 = temp_partition_arxiv(data, year_bound=[-1,2013,2015], proportion=1.0)
data_2013_2015 = data_2013_2015.to(device)

In [19]:
test_loss, test_acc = test(encoder, mlp, data_2013_2015, evaluator)
test_acc_list.append(test_acc)

In [20]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.476 Test Acc: 0.591


In [21]:
encoder_2012_2014_2015, mlp_2012_2014_2015, best_val_loss = continual_adapt(data_2012_2013, [2012,2014,2015], encoder, mlp, device, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Loss: 1.72277 Train Src Loss: 1.04937 Train Tgt Loss: 0.6734 
                 Val Loss: 2.24007 Val Src Loss: 1.59449 Val Tgt Loss: 0.64557
Epoch 2/500 Train Loss: 1.71361 Train Src Loss: 1.14284 Train Tgt Loss: 0.57077 
                 Val Loss: 2.21706 Val Src Loss: 1.58068 Val Tgt Loss: 0.63638
Epoch 3/500 Train Loss: 1.68259 Train Src Loss: 1.11978 Train Tgt Loss: 0.56281 
                 Val Loss: 2.23034 Val Src Loss: 1.59596 Val Tgt Loss: 0.63438
Epoch 4/500 Train Loss: 1.66686 Train Src Loss: 1.09917 Train Tgt Loss: 0.56769 
                 Val Loss: 2.21911 Val Src Loss: 1.58515 Val Tgt Loss: 0.63396
Epoch 5/500 Train Loss: 1.67454 Train Src Loss: 1.10594 Train Tgt Loss: 0.5686 
                 Val Loss: 2.21107 Val Src Loss: 1.58444 Val Tgt Loss: 0.62663
Epoch 6/500 Train Loss: 1.65858 Train Src Loss: 1.09794 Train Tgt Loss: 0.56064 
                 Val Loss: 2.20128 Val Src Loss: 1.59144 Val Tgt L

Epoch 57/500 Train Loss: 1.54431 Train Src Loss: 1.08128 Train Tgt Loss: 0.46303 
                 Val Loss: 2.12513 Val Src Loss: 1.5828 Val Tgt Loss: 0.54233
Epoch 58/500 Train Loss: 1.53494 Train Src Loss: 1.07516 Train Tgt Loss: 0.45978 
                 Val Loss: 2.1492 Val Src Loss: 1.60152 Val Tgt Loss: 0.54768


In [22]:
print(f"Total Val Loss: {best_val_loss}")

Total Val Loss: 2.11966609954834


## 2015-2016 ##

* Test:
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)

In [23]:
data_2015_2017 = temp_partition_arxiv(data, year_bound=[-1,2015,2017], proportion=1.0)
data_2015_2017 = data_2015_2017.to(device)

In [24]:
test_loss, test_acc = test(encoder_2012_2014_2015, mlp_2012_2014_2015, data_2015_2017, evaluator)
test_acc_list.append(test_acc)

In [25]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.655 Test Acc: 0.569


In [26]:
encoder_2014_2016_2017, mlp_2014_2016_2017, best_val_loss = continual_adapt(data_2012_2013, [2014,2016,2017], encoder_2012_2014_2015, mlp_2012_2014_2015, device, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Loss: 1.60251 Train Src Loss: 1.08336 Train Tgt Loss: 0.51916 
                 Val Loss: 2.18959 Val Src Loss: 1.6651 Val Tgt Loss: 0.52448
Epoch 2/500 Train Loss: 1.6787 Train Src Loss: 1.16422 Train Tgt Loss: 0.51448 
                 Val Loss: 2.11771 Val Src Loss: 1.57099 Val Tgt Loss: 0.54672
Epoch 3/500 Train Loss: 1.60331 Train Src Loss: 1.0896 Train Tgt Loss: 0.51371 
                 Val Loss: 2.12416 Val Src Loss: 1.58443 Val Tgt Loss: 0.53973
Epoch 4/500 Train Loss: 1.62159 Train Src Loss: 1.10729 Train Tgt Loss: 0.5143 
                 Val Loss: 2.14295 Val Src Loss: 1.60942 Val Tgt Loss: 0.53353
Epoch 5/500 Train Loss: 1.62386 Train Src Loss: 1.11425 Train Tgt Loss: 0.50962 
                 Val Loss: 2.09815 Val Src Loss: 1.58373 Val Tgt Loss: 0.51442
Epoch 6/500 Train Loss: 1.61067 Train Src Loss: 1.10898 Train Tgt Loss: 0.50169 
                 Val Loss: 2.09456 Val Src Loss: 1.58115 Val Tgt Los

In [27]:
print(f"Total Val Loss: {best_val_loss}")

Total Val Loss: 2.041443347930908


## 2017-2018 ##

* Test:
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)

In [28]:
data_2017_2019 = temp_partition_arxiv(data, year_bound=[-1,2017,2019], proportion=1.0)
data_2017_2019 = data_2017_2019.to(device)

In [29]:
test_loss, test_acc = test(encoder_2014_2016_2017, mlp_2014_2016_2017, data_2017_2019, evaluator)
test_acc_list.append(test_acc)

In [30]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.742 Test Acc: 0.584


In [31]:
encoder_2016_2018_2019, mlp_2016_2018_2019, best_val_loss = continual_adapt(data_2012_2013, [2016,2018,2019], encoder_2014_2016_2017, mlp_2014_2016_2017, device, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Loss: 1.52092 Train Src Loss: 1.07163 Train Tgt Loss: 0.44929 
                 Val Loss: 2.0423 Val Src Loss: 1.61552 Val Tgt Loss: 0.42678
Epoch 2/500 Train Loss: 1.54883 Train Src Loss: 1.11537 Train Tgt Loss: 0.43345 
                 Val Loss: 2.02244 Val Src Loss: 1.58887 Val Tgt Loss: 0.43358
Epoch 3/500 Train Loss: 1.52743 Train Src Loss: 1.08633 Train Tgt Loss: 0.4411 
                 Val Loss: 2.02202 Val Src Loss: 1.59185 Val Tgt Loss: 0.43018
Epoch 4/500 Train Loss: 1.52403 Train Src Loss: 1.08783 Train Tgt Loss: 0.4362 
                 Val Loss: 2.03936 Val Src Loss: 1.59928 Val Tgt Loss: 0.44007
Epoch 5/500 Train Loss: 1.51558 Train Src Loss: 1.07656 Train Tgt Loss: 0.43902 
                 Val Loss: 2.01399 Val Src Loss: 1.58132 Val Tgt Loss: 0.43267
Epoch 6/500 Train Loss: 1.51021 Train Src Loss: 1.07439 Train Tgt Loss: 0.43582 
                 Val Loss: 2.00509 Val Src Loss: 1.58161 Val Tgt Lo

In [32]:
print(f"Total Val Loss: {best_val_loss}")

Total Val Loss: 1.9615998268127441


## 2019-2020 ##

In [33]:
data_2019_2021 = temp_partition_arxiv(data, year_bound=[-1,2019,2021], proportion=1.0)
data_2019_2021 = data_2019_2021.to(device)

In [34]:
test_loss, test_acc = test(encoder_2016_2018_2019, mlp_2016_2018_2019, data_2019_2021, evaluator)
test_acc_list.append(test_acc)

In [35]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.792 Test Acc: 0.609


In [36]:
print(test_acc_list)

[0.5907802649083232, 0.5685134277860012, 0.5839074178880194, 0.6085426825504598]


In [37]:
print(sum(test_acc_list) / len(test_acc_list))

0.587935948283201
